# Preparar clasificación de noticias RPP
Este cuaderno:
1. Cargamos las 50 noticias desde `data/rpp_source.json`.
2. Generamos el bloque numerado para pedir clasificación a un LLM (0-World, 1-Sports, 2-Business, 3-Science/Tech).
3. Guardamos el resultado clasificado en `data/rpp_classified.json`.

In [13]:
# === Celda 0: sincronizar ruta del repo en Colab ===
import os, sys

In [14]:
# 1️. Borrar clonaciones anteriores
!rm -rf /content/News_Classification-lab_LovatonValeria_PizarroSebastian

# 2️. Clonar el repo completo desde GitHub
!git clone https://github.com/ValLovaton/News_Classification-lab_LovatonValeria_PizarroSebastian
%cd /content/News_Classification-lab_LovatonValeria_PizarroSebastian


# 3️⃣ Mostrar estructura
print("\n📂 Estructura inicial:")
!ls -R | head -40

# 4️⃣ Asegurar ruta base y carpeta data
sys.path.append(os.path.abspath("."))
os.makedirs("data", exist_ok=True)

print("\n✅ Ubicación actual:", os.getcwd())


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: could not create work tree dir 'News_Classification-lab_LovatonValeria_PizarroSebastian': No such file or directory
[Errno 2] No such file or directory: '/content/News_Classification-lab_LovatonValeria_PizarroSebastian'
/content/News_Classification-lab_LovatonValeria_PizarroSebastian

📂 Estructura inicial:
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
.:


FileNotFoundError: [Errno 2] No such file or directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
import json, pandas as pd, os

1. Cargar las 50 noticias del RSS
with open('data/rpp_source.json', 'r', encoding='utf-8') as f:
    news = json.load(f)
df = pd.DataFrame(news)
print(f'Noticias cargadas: {len(df)}')
df.head(2)

✅ Archivo guardado correctamente en: data/rpp_classified.json
Primeras filas del archivo:


,title,description,link,date_published,label_llm
0,Ollanta Humala se pronuncia tras anulación de ...,"El expresidente Humala, sentenciado a 15 años ...",https://rpp.pe/politica/judiciales/ollanta-hum...,"Mon, 20 Oct 2025 22:19:10 -0500",2
1,"Vecina de San Isidro considera que hubo ""tardí...","Cynthia Yamamoto, vocera de 'Peruanos de a pie...",https://rpp.pe/lima/actualidad/san-isidro-veci...,"Mon, 20 Oct 2025 22:01:43 -0500",0
2,Riveros acabó en el piso: así fue superado el ...,Juan Lucumí de Ayacucho FC silenció el Estadio...,https://rpp.pe/futbol/descentralizado/ver-gol-...,"Mon, 20 Oct 2025 21:30:33 -0500",0
3,"Partidos de hoy, martes 21 de octubre del 2025...",Fútbol EN VIVO | Horarios y canales de TV para...,https://rpp.pe/futbol/futbol-mundial/partidos-...,"Mon, 20 Oct 2025 22:00:58 -0500",1
4,Lluvias en Huancayo: ¿qué daños provocaron en ...,"Los distritos de El Tambo, Chilca, Sapallanga ...",https://rpp.pe/peru/junin/lluvias-en-huancayo-...,"Mon, 20 Oct 2025 21:05:01 -0500",3
